<a href="https://www.kaggle.com/code/dzung271828/ouro-trace?scriptVersionId=288557226" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Setup libraries

In [11]:
!uv pip install --upgrade pip
!uv pip uninstall transformers tokenizers accelerate -q

!uv pip install "transformers==4.56.0" "protobuf==5.29.3" -q
!uv pip install torch datasets -q
!uv pip install pandas matplotlib seaborn tqdm wandb pyyaml
!uv pip install bitsandbytes accelerate optimum lm_eval
# !uv pip install -r requirements.txt
!uv pip install --force-reinstall --no-cache-dir "numpy<2.0"

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 47ms
Audited 1 package in 0.27ms
Using Python 3.12.12 environment at: /usr
Audited 6 packages in 139ms
Using Python 3.12.12 environment at: /usr
Resolved 95 packages in 312ms
Installed 1 package in 6ms
 + accelerate==1.12.0
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 109ms
Prepared 1 package in 411ms
Uninstalled 1 package in 19ms
Installed 1 package in 16ms
 ~ numpy==1.26.4


# Suppress warnings

In [12]:
# Suppress warnings for clean output
import warnings
import os

warnings.filterwarnings("ignore", category=UserWarning)
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
print("✅ Packages installed successfully!")

✅ Packages installed successfully!


# Install Libraries

In [13]:
"Built-in libraries"
import re
import sys
import gc
import time
import json
import hashlib
import glob
import zipfile
from io import StringIO
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
import yaml
import logging
import random

"Deep learning and NLP libraries"
import torch
import torch.nn.functional as F
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
    logging as hf_logging,
)

"Data processing libraries"
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import wandb
from tqdm.auto import tqdm
from IPython import get_ipython

# Configure logging
logging.getLogger("ContinuousBatchingLogger").setLevel(logging.ERROR)
hf_logging.set_verbosity_error()


print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
!nvidia-smi

Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch Version: 2.9.0+cu126
CUDA Available: True
CUDA Version: 12.6
Sat Dec 27 04:07:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P0             30W /   70W |     150MiB /  15360MiB |      0%      Default |
|   

In [14]:
import os


def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")

    return base_data_path, base_output_path, environment_name


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/


# Setup WANDB

In [15]:
import os
import wandb

if "colab" in ENV_NAME:
    from google.colab import userdata

    try:
        # Ensure 'WANDB_API_KEY' is the exact name in your Colab Secrets (the key icon)
        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Colab Secrets: {e}")

# 2. Check if running in Kaggle
elif "kaggle" in ENV_NAME:
    try:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        wandb_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Kaggle Secrets: {e}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Config input/output path and clone latest repo

In [16]:
# Clone the latest github repo version
%cd {OUTPUT_PATH}
torch.cuda.empty_cache()
!rm -rf OuroTrace

/content


In [17]:
!git clone --branch claude https://github.com/dzungphieuluuky/OuroTrace.git
%cd OuroTrace

Cloning into 'OuroTrace'...
remote: Enumerating objects: 2422, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 2422 (delta 173), reused 177 (delta 58), pack-reused 2118 (from 2)
Receiving objects: 100% (2422/2422), 7.46 MiB | 14.02 MiB/s, done.
Resolving deltas: 100% (1530/1530), done.
/content/OuroTrace


# Run Benchmark

In [ ]:
from src.config_loader import load_config_from_json, post_process_config
from src.new_refactored_runner import run_experiment
from src.evaluation_analysis import analyze_experiment_results

def set_all_seeds(seed):
    random.seed(seed)                          # Python random
    os.environ['PYTHONHASHSEED'] = str(seed)  # Python hash seed
    np.random.seed(seed)                      # NumPy
    torch.manual_seed(seed)                   # PyTorch CPU & GPU

    # Additional GPU-specific settings
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)      # For multi-GPU

set_all_seeds(1415)
# 1. Load Configuration from JSON
config = load_config_from_json("configs/ouro_1.4b_thinking.json")

# 2. Post-process (Convert 'torch.float16' string to object, generate timestamps)
config = post_process_config(config)

config["INFERENCE_STEPS"] = [9]
config["OPTIMIZATION"]["enable_batch"] = False
config["EVAL_SETTINGS"]["calculate_perplexity"] = False
# config["DATA"]["n_ary"]["num_samples_per_level"] = 0
# config["DATA"]["p_hop"]["num_samples_per_level"] = 0
config["DATA"]["igsm"]["num_samples"] = 0
config["DATA"]["reasoning_primitives"]["num_samples"] = 0

# 4. Execute
print("🚀 Starting Experiment...")
try:
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()
except:
    pass

try:
    simple_reasoning_results, ppl_results, primitives_results, benchmark_results = run_experiment(config)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

🚀 Starting Experiment...
Initializing W&B (timeout: 30s)...


W&B initialized successfully

EXPERIMENT CONFIGURATION
Model Path: ByteDance/Ouro-1.4B-Thinking
UT Steps to Test: [9]
Data Type: torch.bfloat16
4-bit Quantization: False
Torch Compile: False
Max Batch Size: 8
Max New Tokens: 16
Batching: False
Calculate Perplexity: False
Early Exit: 1.0

Quality monitor initialized:
    Garbage threshold: 30%
    Example similarity threshold: 85%
    Min samples before check: 10

LOADING TEST DATASETS
Generating new test datasets...
Generated test datasets successfully

Dataset Summary:
   n_ary       :  500 samples
   p_hop       :  300 samples
   igsm        :    0 samples

✅ Configuration saved to ../results_20251227_041316_UT_9/config.json
✅ Task templates saved to ../results_20251227_041316_UT_9/task_templates.json

EXPERIMENT 1/1: UT Steps = 9


LOADING MODEL CONFIGURATION
Model Path: ByteDance/Ouro-1.4B-Thinking
Requested UT Steps: 9
Data Type: torch.bfloat16
4-bit Quantization: False
Torch Compile: False

Base config loaded
   Original UT steps

   n_ary:   0%|          | 0/500 [00:00<?, ?it/s]

    test_input        full_response  generated_tokens
0  871 + 556 =  [FINAL]  1427 [END]                 8
    test_input        full_response  generated_tokens
0  888 + 759 =  [FINAL]  1647 [END]                 8
    test_input        full_response  generated_tokens
0  670 + 418 =  [FINAL]  1088 [END]                 8
    test_input        full_response  generated_tokens
0  661 + 544 =  [FINAL]  1205 [END]                 8
    test_input        full_response  generated_tokens
0  328 + 902 =  [FINAL]  1230 [END]                 8
    test_input       full_response  generated_tokens
0  696 + 243 =  [FINAL]  939 [END]                 7
    test_input        full_response  generated_tokens
0  810 + 357 =  [FINAL]  1167 [END]                 8
    test_input       full_response  generated_tokens
0  191 + 780 =  [FINAL]  971 [END]                 7


In [19]:
import os
import glob
import zipfile
from typing import List
def find_result_folders(base_path: str) -> List[str]:
    """
    Return a list of absolute paths to all directories under `base_path`
    whose names start with 'results_'.
    """
    pattern = os.path.join(base_path, "results_*")
    # glob returns both files and directories; filter to directories only
    return [p for p in glob.glob(pattern) if os.path.isdir(p)]
def zip_folder(folder_path: str, output_base_path: str) -> bool:
    """
    Zip the contents of `folder_path` into a file named
    <folder_name>.zip` inside `output_base_path`.

    Returns True on success, False otherwise.
    """
    folder_name = os.path.basename(folder_path)
    zip_path = os.path.join(output_base_path, f"{folder_name}.zip")
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(
            zip_path, mode="w", compression=zipfile.ZIP_DEFLATED
        ) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    # Preserve relative path inside the zip
                    arcname = os.path.relpath(full_path, os.path.dirname(folder_path))
                    zipf.write(full_path, arcname)
        print(f"   ✅ Created ZIP: {os.path.basename(zip_path)}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False
def zip_stats_results_folders(output_base_path: str) -> None:
    """
    Main driver: locate all result folders and zip each one.
    """
    # Ensure the output directory exists
    os.makedirs(output_base_path, exist_ok=True)
    result_folders = find_result_folders(output_base_path)
    if not result_folders:
        print(f"⚠️ No folders starting with 'results_' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = 0
    for folder in result_folders:
        if zip_folder(folder, output_base_path):
            successful += 1
    print(
        f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s)."
    )
if __name__ == "__main__":
    try:
        # Prefer an environment variable; fall back to a global if defined
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        # The script expects a sub‑folder named 'OuroTrace' under OUTPUT_PATH
        target_path = os.path.join(output_root, "")
        zip_stats_results_folders(target_path)
    except Exception as e:
        print(f"❌ An error occurred: {e}")

🔍 Found 2 result folder(s) to zip.
   -> Zipping folder: results_20251227_030057_UT_4...
   ✅ Created ZIP: results_20251227_030057_UT_4.zip
   -> Zipping folder: results_20251227_040751_UT_12...
   ✅ Created ZIP: results_20251227_040751_UT_12.zip

✅ DONE! Successfully zipped 2 out of 2 folder(s).


In [20]:
df_simple = pd.DataFrame(simple_reasoning_results)
df_ppl = pd.DataFrame(ppl_results)

print("Final Inspection:\n")
print("Top 20 Accuracy Report:\n")
print(df_simple.head(20))
print(f"Full Response:\n")
print(df_simple["full_response"])
print("Perplexity Report:\n")
print(df_ppl.head(20))

Final Inspection:

Top 20 Accuracy Report:

   task_type difficulty   test_input expected_answer prediction  is_correct  \
0      n_ary      2_ops  871 + 556 =            1427       1427        True   
1      n_ary      2_ops  888 + 759 =            1647       1647        True   
2      n_ary      2_ops  670 + 418 =            1088       1088        True   
3      n_ary      2_ops  661 + 544 =            1205       1205        True   
4      n_ary      2_ops  328 + 902 =            1230       1230        True   
5      n_ary      2_ops  696 + 243 =             939        939        True   
6      n_ary      2_ops  810 + 357 =            1167       1167        True   
7      n_ary      2_ops  191 + 780 =             971        971        True   
8      n_ary      2_ops  933 + 652 =            1585       1585        True   
9      n_ary      2_ops  995 + 551 =            1546       1546        True   
10     n_ary      2_ops  227 + 885 =            1112       1112        True   
11     n

In [21]:
print(df_simple[["full_response", "generated_tokens"]])

          full_response  generated_tokens
0   [FINAL]  1427 [END]                 8
1   [FINAL]  1647 [END]                 8
2   [FINAL]  1088 [END]                 8
3   [FINAL]  1205 [END]                 8
4   [FINAL]  1230 [END]                 8
5    [FINAL]  939 [END]                 7
6   [FINAL]  1167 [END]                 8
7    [FINAL]  971 [END]                 7
8   [FINAL]  1585 [END]                 8
9   [FINAL]  1546 [END]                 8
10  [FINAL]  1112 [END]                 8
11  [FINAL]  1308 [END]                 8
12  [FINAL]  1461 [END]                 8
13   [FINAL]  516 [END]                 7
14  [FINAL]  1221 [END]                 8
